In [1]:
import os
import sys

sys.path.append("../python")

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from data import Dataloader
from models import RandomExampleModel
import metrics

In [3]:
# Loading the data
X = np.load("../data/dataset_4/X.npy") # Specify path to the dataset you want to load
Y = np.load("../data/dataset_4/Y.npy")


In [4]:
X_re = X
Y_re = Y

In [31]:
from gurobipy import *
import math
# Instanciation du modèle
m = Model("IntuitionV0")

PRECISION = 0.001
#Nombre de features
N_CRITERIA = len(X_re[0])

#Nombre de segment/breakpoints dans toutes les fonctions de score, possible de l'adapter pour chaque feature 
L = 5
#Nombre d'occurence dans le dataset/produits traités 
N_COMPARAISON = len(X_re)


#Nombre de clusters 
K=2

#Majorant contrainte 
M=4

#à définir clairement : 
criteria = [[[(m.addVar(name=f"u_{j}{l}{k}")) for k in range(K)] for l in range(L+1)]for j in range(N_CRITERIA) ]

sigma_Xplus = [[(m.addVar(name=f"sigma+{i}{k}")) for k in range(K) ] for i in range(N_COMPARAISON)]
sigma_Xminus = [[(m.addVar(name=f"sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yplus = [[(m.addVar(name=f"sigma+{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yminus = [[(m.addVar(name=f"sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]


z_binary = [[m.addVar(name=f"z_{i}_{k}",vtype=GRB.BINARY)for k in range(K) ] for i in range(N_COMPARAISON)]

#hyperparamètre pour le modèle
epsilon = PRECISION
# maj du modèle
m.update()


min_int_X = X_re.min(axis=0)
min_int_Y = Y_re.min(axis=0)
mins = np.min(np.vstack((min_int_X, min_int_Y)), axis = 0)

max_int_X = X_re.max(axis=0)
max_int_Y = Y_re.max(axis=0)
maxs = np.max(np.vstack((max_int_X, max_int_Y)), axis = 0)


#donne un chiffre entre 0 et L (ici 5) qui donne l'appartenance à l'un des 6 segment possible 
def segment(i, j, X):
    x = X[i][j]
    return math.floor(L * (x - mins[j]) / (maxs[j] - mins[j]))
r 
#retourne l'abscisse de gauche du tronçon l 
def x_seg(j, seg):
    return mins[j] + seg * (maxs[j] - mins[j]) / L

#calcule l'ordonéne de x sur le tronçon du feaure considéré
def u(i,j,k,X,eval : bool = False):
    get_val = (lambda v: v.X) if eval else (lambda v: v)
    x = X[i][j]
    
    if x == maxs[j]:
        return get_val(criteria[j][L][k]) 
    
    else: 
        segm = segment(i, j, X)
        x_segm = x_seg(j, segm)
        #print(r"x_segm", x_segm)
        x_segm_1 = x_seg(j, segm +1)
        
        #print(r"x_segm_1", x_segm_1)
        S=0
        S = criteria[j][segm][k]
        slope = (x - x_segm)/(x_segm_1 - x_segm)
        width = criteria[j][segm+1][k] - criteria[j][segm][k]
        S =+ slope*width
        return S


#définissons la somme des fonctions de score pour un produit (de rang i)
def s(i, X, k, eval : bool = False):
    if not eval : 
        return quicksum(u(i,j,k,X,False) for j in range(N_CRITERIA))
    else : 
        return sum(u(i,j,k,X,False) for j in range(N_CRITERIA))




#Contrainte cahque paire doit être représentée par un cluster
for i in range(N_COMPARAISON):
    m.addConstr(z_binary[i][0] + z_binary[i][1] >= 1)

#Contrainte pour choisir le cluster
for k in range(K): 
    for i in range(N_COMPARAISON) :
        m.addConstr((s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k]) <= (M*z_binary[i][k]+ epsilon))
        m.addConstr(M*(1-z_binary[i][k])<= (s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k]) - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k])+2*epsilon)

# Constraint for sigma_Xminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xminus[i][k] >= 0)

# Constraint for sigma_Xplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xplus[i][k] >= 0)

# Constraint for sigma_Yminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yminus[i][k] >= 0)

# Constraint for sigma_Yplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yplus[i][k] >= 0)


# contrainte 3
for k in range(K) :
    for j in range(N_CRITERIA):
        for l in range(L):
            m.addConstr(criteria[j][l+1][k] - criteria[j][l][k] >= epsilon)

# contrainte 4
for k in range(K):
        m.addConstr(quicksum(criteria[j][L][k] for j in range(N_CRITERIA)) ==1)
    
#contrainte 5
for k in range(K):    
    for j in range(N_CRITERIA): 
        m.addConstr(criteria[j][0][k] == 0)    

m.update()
# Flatten the lists of lists into a single list
sigma_Xplus_flat = [var for sublist in sigma_Xplus for var in sublist]
sigma_Xminus_flat = [var for sublist in sigma_Xminus for var in sublist]
sigma_Yminus_flat = [var for sublist in sigma_Yminus for var in sublist]
sigma_Yplus_flat = [var for sublist in sigma_Yplus for var in sublist]

# Objective function
m.setObjective(sum(sigma_Xplus_flat) + sum(sigma_Xminus_flat) + sum(sigma_Yminus_flat) + sum(sigma_Yplus_flat), GRB.MINIMIZE)


# Résolution du PL
m.optimize()


for i in range(N_COMPARAISON):
    for k in range(K): 
        print(z_binary[i][k])
for i in range(N_COMPARAISON):
    print(z_binary[i][0])

# for j in range(N_CRITERIA):
#     for l in range(L):
#         print(criteria[j][l][0], criteria[j][l][1] )

NameError: name 'r' is not defined

In [33]:
from gurobipy import *
import math
# Instanciation du modèle
m = Model("IntuitionV0")

PRECISION = 0.0001
#Nombre de features
N_CRITERIA = len(X_re[0])

#Nombre de segment/breakpoints dans toutes les fonctions de score, possible de l'adapter pour chaque feature 
L = 5
#Nombre d'occurence dans le dataset/produits traités 
N_COMPARAISON = len(X_re)


#Nombre de clusters 
K=2

#Majorant contrainte 
M=4

#à définir clairement : 


criteria = [[[(m.addVar(name=f"u_{j}{l}{k}")) for k in range(K)] for l in range(L+1)] for j in range(N_CRITERIA) ]

sigma_Xplus = [[(m.addVar(name=f"x_sigma+{i}{k}")) for k in range(K) ] for i in range(N_COMPARAISON)]
sigma_Xminus = [[(m.addVar(name=f"x_sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yplus = [[(m.addVar(name=f"y_sigma+{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]
sigma_Yminus = [[(m.addVar(name=f"y_sigma-{i}{k}")) for k in range(K)] for i in range(N_COMPARAISON)]


z_binary = [[m.addVar(name=f"z_{i}_{k}",vtype=GRB.BINARY) for k in range(K) ] for i in range(N_COMPARAISON)]

#hyperparamètre pour le modèle
epsilon = PRECISION
# maj du modèle
m.update()

min_int_X = X_re.min(axis=0)
min_int_Y = Y_re.min(axis=0)
mins = np.min(np.vstack((min_int_X, min_int_Y)), axis = 0)

max_int_X = X_re.max(axis=0)
max_int_Y = Y_re.max(axis=0)
maxs = np.max(np.vstack((max_int_X, max_int_Y)), axis = 0)

#donne un chiffre entre 0 et L (ici 5) qui donne l'appartenance à l'un des 6 segment possible 
def segment(i, j, X):
    x = X[i][j]
    return math.floor(L * (x - mins[j]) / (maxs[j] - mins[j]))

#retourne l'abscisse de gauche ? du tronçon l 
#appelé xik dans question 3
def x_seg(j, seg):
    return mins[j] + seg * (maxs[j] - mins[j]) / L

#calcule la valeur si(xij)
#cad le score partiel pour un feature donné 
# ?? on obtient le score à partir d'une forme ressemblant au développement limité autour de la valeur de la feature sur le tronçon

def u(i,j,k,X,eval : bool = False):
    get_val = (lambda v: v.X) if eval else (lambda v: v)
    x = X[i][j]
    
    if x == maxs[j]:
        return get_val(criteria[j][-1][k]) 
    else:
        segm = segment(i, j, X)
        x_segm = x_seg(j, segm)
        x_segm_1 = x_seg(j, segm +1)    
        slope = (x - x_segm)/(x_segm_1 - x_segm)
        width = criteria[j][segm+1][k] - criteria[j][segm][k]
        S =  slope*width + criteria[j][segm][k]
    return S


#définissons la somme des fonctions de score pour un produit (de rang i)
def s(i, X, k, eval : bool = False):
    if not eval : 
        return quicksum(u(i,j,k,X,False) for j in range(N_CRITERIA))
    else : 
        return sum(u(i,j,k,X,False) for j in range(N_CRITERIA))
    
#Au moins un des deux clusters doit prédire l'instance 
for i in range(N_COMPARAISON):
    m.addConstr(z_binary[i][0] + z_binary[i][1] >= 1)


#Ajout des contraintes
for k in range(K):
    for i in range(N_COMPARAISON) :
        m.addConstr(s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k] - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k])<= M*z_binary[i][k]+ epsilon)
        m.addConstr(M*(1-z_binary[i][k])<= s(i, X_re, k, False) - sigma_Xplus[i][k] + sigma_Xminus[i][k] - (s(i, Y_re, k, False) - sigma_Yplus[i][k] + sigma_Yminus[i][k]) + 2*epsilon)

# Constraint for sigma_Xminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xminus[i][k] >= 0)

# Constraint for sigma_Xplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Xplus[i][k] >= 0)

# Constraint for sigma_Yminus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yminus[i][k] >= 0)

# Constraint for sigma_Yplus
for k in range(K):
    for i in range(N_COMPARAISON):
        m.addConstr(sigma_Yplus[i][k] >= 0)

# contrainte 3
for k in range(K) :
    for l in range(L-1):
        for j in range(N_CRITERIA):
            m.addConstr(criteria[j][l+1][k] - criteria[j][l][k] >= 0)


# contrainte 4
for k in range(K):
        m.addConstr(quicksum(criteria[j][L][k] for j in range(N_CRITERIA)) ==1)

# contrainte 5
for k in range(K): 
    for j in range(N_CRITERIA): 
        m.addConstr(criteria[j][0][k] == 0)

m.update()


somme = 0
for i in range(N_COMPARAISON):
    for k in range(K):
        somme+= sigma_Xplus[i][k]+sigma_Xminus[i][k]+sigma_Yplus[i][k]+sigma_Yminus[i][k]

m.setObjective(somme, GRB.MINIMIZE)

m.optimize()
# for j in range(N_CRITERIA):
#     for l in range(L+1):
#         print(criteria[j][l][0], criteria[j][l][1] )

for i in range(N_COMPARAISON):
    print(z_binary[i][k])
    print(z_binary[i][0])

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.1.0 23B74)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26042 rows, 20048 columns and 163564 nonzeros
Model fingerprint: 0xf33e0cfa
Variable types: 16048 continuous, 4000 integer (4000 binary)
Coefficient statistics:
  Matrix range     [1e-04, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 4e+00]
Presolve removed 22029 rows and 16028 columns
Presolve time: 0.24s
Presolved: 4013 rows, 4020 columns, 53818 nonzeros
Variable types: 4020 continuous, 0 integer (0 binary)

Root relaxation: objective 8.264470e+01, 1302 iterations, 0.25 seconds (0.50 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      82.6447000   82.6

In [ ]:
for k in range(K):
    for i in range(N_COMPARAISON):
        print(k)
        print(s(i, X_re, k,False).getValue() > s(i, Y_re, k,False).getValue())

In [26]:
for k in range(K):
    for i in range(N_COMPARAISON):
        print(k)
        print(tot_X(i, k, X_re).getValue() > tot_Y(i, k, Y_re).getValue())

0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True
0
True